In [1]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# -------------- Dataset Definition --------------
class SequenceDataset(Dataset):
    def __init__(self, file_paths, n_mfcc=None):
        self.file_paths = file_paths
        self.n_mfcc = n_mfcc

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        arr = np.load(path)
        # Handle MFCC reshape
        if self.n_mfcc and arr.ndim == 1:
            T = arr.size // self.n_mfcc
            arr = arr.reshape(self.n_mfcc, T)
        # arr: (feat_dim, time_steps) → (time_steps, feat_dim)
        x = torch.tensor(arr, dtype=torch.float32).transpose(0, 1)
        # class label from filename
        fname = os.path.basename(path)
        class_id = int(fname.split('-')[1].split('.')[0])
        y = torch.tensor(class_id, dtype=torch.long)
        return x, y

# -------------- Model Definitions --------------
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=3, num_classes=10):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc   = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)       # h_n: (num_layers, B, hidden_dim)
        return self.fc(h_n[-1])          # use last layer’s hidden state

class CNNClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # conv stack
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool2d(2)
        # instead of hard-coding dims, force output to 1×1
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

        # now every sample is reduced to a (B, 64, 1, 1) tensor → 64 features
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # x: (B, T, D) → (B, 1, D, T)
        x = x.permute(0, 2, 1).unsqueeze(1)
        x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        # global pooling
        x = self.adaptive_pool(x)         # → (B, 64, 1, 1)
        x = x.view(x.size(0), -1)         # → (B, 64)
        x = F.relu(self.fc1(x))           # → (B, 128)
        return self.fc2(x)                # → (B, num_classes)

# -------------- Training & Evaluation --------------
def train_model(model, loader, crit, opt, device):
    model.train()
    tot_loss = 0.0
    for x,y in tqdm(loader, desc="Train"):
        x, y = x.to(device), y.to(device)
        opt.zero_grad()
        loss = crit(model(x), y)
        loss.backward()
        opt.step()
        tot_loss += loss.item() * x.size(0)
    return tot_loss / len(loader.dataset)

@torch.no_grad()
def eval_model(model, loader, device):
    model.eval()
    correct = 0
    total   = 0
    for x,y in tqdm(loader, desc="Eval"):
        x, y = x.to(device), y.to(device)
        preds = model(x).argmax(dim=1)
        correct += (preds == y).sum().item()
        total   += y.size(0)
    return correct/total if total>0 else 0.0

# -------------- Main Experiment Loop --------------
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    DATASETS = {
        'STFT': [
            r"D:\Aliasing3\Processed_Files\DS_ESC",
                 # r"D:\Aliasing3\Processed_Files\DS_U8K"
        ],
        'MFCC': [
            r"D:\Aliasing3\Processed_Files\DS_ESC_MFCC",
                 # r"D:\Aliasing3\Processed_Files\DS_U8K_MFCC"
        ]
    }
    frequencies = [1000, 2000, 4000, 8000, 16000, 22000]
    subsets     = ['filtered', 'unfiltered']
    batch_size  = 4
    epochs      = 50
    results     = []
    os.makedirs('checkpoints_sep', exist_ok=True)
    for feat_type, roots in DATASETS.items():
        n_mfcc = 20 if feat_type=='MFCC' else None

        for root in roots:
            num_classes = 50 if 'ESC' in root else 10

            for freq in frequencies:
                for model_name in ['LSTM','CNN']:
                    for subset in subsets:

                        # collect only this subset’s files
                        tr_glob = os.path.join(root, str(freq), 'train',   subset, '*.npy')
                        va_glob = os.path.join(root, str(freq), 'validation', subset, '*.npy')
                        te_glob = os.path.join(root, str(freq), 'test', subset, '*.npy')
                        train_files = glob.glob(tr_glob)
                        val_files   = glob.glob(va_glob)
                        test_files   = glob.glob(te_glob)

                        print(f"\n>>> {feat_type} | {root} | {freq}Hz | {model_name} | {subset}")
                        print("  #train:", len(train_files), "  #val:", len(val_files))
                        if len(train_files)==0 or len(val_files)==0:
                            print("  Skipping—no data.")
                            continue

                        # DataLoaders
                        train_ds = SequenceDataset(train_files, n_mfcc=n_mfcc)
                        val_ds   = SequenceDataset(val_files,   n_mfcc=n_mfcc)
                        test_ds   = SequenceDataset(test_files,   n_mfcc=n_mfcc)
                        train_ld = DataLoader(train_ds, batch_size, shuffle=True)
                        val_ld   = DataLoader(val_ds,   batch_size)
                        test_ld   = DataLoader(test_ds,   batch_size)

                        # Model
                        if model_name=='LSTM':
                            # infer input dim
                            sample_x,_ = next(iter(train_ld))
                            input_dim  = sample_x.shape[2]
                            model = LSTMClassifier(input_dim, hidden_dim=128, num_layers=3, num_classes=num_classes)
                        else:
                            model = CNNClassifier(num_classes=num_classes)
                        model.to(device)

                        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
                        criterion = nn.CrossEntropyLoss()
 
                        # NEW: prepare best‐validation tracking
                        best_val_acc = 0.0
                        best_state   = None
                        epoch_records = []
                        
                        # Train
                        for ep in range(1, epochs+1):
                            loss    = train_model(model, train_ld, criterion, optimizer, device)
                            val_acc = eval_model(model, val_ld, device)
                            print(f" Epoch {ep}:  val_acc={val_acc:.3%}")
 
                            # NEW: save every epoch’s weights
                            ckpt_path = f'checkpoints_sep/{model_name}_{feat_type}_{freq}Hz_{subset}_ep{ep:02d}.pt'
                            torch.save(model.state_dict(), ckpt_path)
 
                            # NEW: keep best‐so‐far
                            if val_acc > best_val_acc:
                                best_val_acc = val_acc
                                # clone to avoid GPU/overwrite issues
                                best_state = {k: v.cpu().clone() for k,v in model.state_dict().items()}
                            epoch_records.append((ep, val_acc))

                        # NEW: after all epochs, reload best weights
                        model.load_state_dict(best_state)
                        model.to(device)
                        print(f"Loaded best val model @ acc={best_val_acc:.3%}")

                        # Eval
                        test_acc = eval_model(model, test_ld, device)
                        print(f"  ▶ test_acc={test_acc:.3%}")

                        # Log
                        results.append({
                            'feat_type':       feat_type,
                            'dataset':         os.path.basename(root),
                            'freq':            freq,
                            'model':           model_name,
                            'subset':          subset,
                            'best_val_acc':    best_val_acc,
                            'test_acc':        test_acc
                        })

    # Save all results
    df = pd.DataFrame(results)
    df.to_csv('difficulty_separate_by_subset_esc.csv', index=False)
    print("\nWrote difficulty_by_subset_esc.csv")



>>> STFT | D:\Aliasing3\Processed_Files\DS_ESC | 1000Hz | LSTM | filtered
  #train: 1503   #val: 191


Eval: 100%|████████████████████████████████████████████████████████████████████████████| 48/48 [00:02<00:00, 18.04it/s]


 Epoch 1:  val_acc=3.665%


Eval: 100%|███████████████████████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 137.63it/s]


 Epoch 2:  val_acc=12.565%


...

 Epoch 49:  val_acc=62.032%


Eval: 100%|███████████████████████████████████████████████████████████████████████████| 47/47 [00:00<00:00, 177.86it/s]


 Epoch 50:  val_acc=66.310%
Loaded best val model @ acc=66.845%


Eval: 100%|███████████████████████████████████████████████████████████████████████████| 49/49 [00:00<00:00, 190.98it/s]


  ▶ test_acc=68.718%

Wrote difficulty_by_subset_esc.csv
